<a href="https://colab.research.google.com/github/present42/PyTorchPractice/blob/main/JAX_Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

In [2]:
key = random.key(0)
x = random.normal(key, (10,))
print(x)

[-0.3721109   0.26423115 -0.18252768 -0.7368197  -0.44030377 -0.1521442
 -0.67135346 -0.5908641   0.73168886  0.5673026 ]


In [3]:
size = 3000
x = random.normal(key, (size, size), dtype=jnp.float32)
%timeit jnp.dot(x, x.T).block_until_ready()

13.4 ms ± 2.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
type(x)

jaxlib.xla_extension.ArrayImpl

In [6]:
import numpy as np

x = np.random.normal(size=(size, size)).astype(np.float32)
%timeit jnp.dot(x, x.T).block_until_ready()

38.9 ms ± 1.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


slower because it has to transfer data to GPU ever y time

In [7]:
from jax import device_put

x = np.random.normal(size=(size, size)).astype(np.float32)
# the output of device_put() still acts like an NDArray
# it only copies values back to CPU when they're needed for printing
# plotting, saving to disk, branching, etc
x = device_put(x)
%timeit jnp.dot(x, x.T).block_until_ready()

16.9 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
def selu(x, alpha=1.67, lmbda=1.05):
  return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = random.normal(key, (1000000,))
%timeit selu(x).block_until_ready()

849 µs ± 90.9 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()

144 µs ± 11.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
def sum_logistic(x):
  return jnp.sum(1.0 / (1.0 + jnp.exp(-x)))

x_small = jnp.arange(3.)
derivative_fn = grad(sum_logistic)
print(derivative_fn(x_small))

[0.25       0.19661194 0.10499357]


In [11]:
def first_finite_differences(f, x):
  eps = 1e-3
  return jnp.array([(f(x + eps * v) - f(x - eps * v)) / (2 * eps) for v in jnp.eye(len(x))])

In [12]:
print(first_finite_differences(sum_logistic, x_small))

[0.24998187 0.1965761  0.10502338]


In [13]:
from jax import jacfwd, jacrev
def hessian(fun):
  return jit(jacfwd(jacrev(fun)))

## Auto-vectorization with `vmap()`

In [14]:
mat = random.normal(key, (150, 100))
batched_x = random.normal(key, (10, 100))

def apply_matrix(v):
  return jnp.dot(mat, v)

In [15]:
def naively_batched_apply_matrix(v_batched):
  return jnp.stack([apply_matrix(v) for v in v_batched])

print('Naively batched')
%timeit naively_batched_apply_matrix(batched_x).block_until_ready()

Naively batched
3.09 ms ± 398 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
@jit
def batched_apply_matrix(v_batched):
  return jnp.dot(v_batched, mat.T)

print('Manually batched')
%timeit batched_apply_matrix(batched_x).block_until_ready()

Manually batched
154 µs ± 33.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [23]:
@jit
def vmap_batched_apply_matrix(v_batched):
  return vmap(apply_matrix)(v_batched)

print('Auto-vectorized with vmap')
%timeit vmap_batched_apply_matrix(batched_x).block_until_ready()

Auto-vectorized with vmap
123 µs ± 13.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
